week 3 of capstone course

In [1]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup 

import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json

!pip install geopy
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!pip install folium
import folium 
!pip install geocoder
import geocoder 

     |████████████████████████████████| 92kB 15.0MB/s eta 0:00:01
     |████████████████████████████████| 102kB 17.1MB/s ta 0:00:01


PART ONE

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_='sortable')
row = [] 
for tr in table.find_all('tr'):                                          
    if tr.find_all('th') == []:                                           
        row.append([td.get_text(strip=True) for td in tr.find_all('td')]) 
column_names = ['Postcode', 'Borough', 'Neighborhood']
neighborhoods_raw = pd.DataFrame(row, columns=column_names)
neighborhoods_raw.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


then i will clean the datatable and drop the rows with "not assigned"

In [5]:
drop_index = neighborhoods_raw[neighborhoods_raw['Borough'] == 'Not assigned'].index 
neighborhoods = neighborhoods_raw.drop(drop_index, axis=0)                           
neighborhoods.reset_index(drop=True, inplace=True)
neighborhood_na = neighborhoods[neighborhoods['Neighborhood'] == 'Not assigned'].index 
neighborhoods.iloc[neighborhood_na, 2] = neighborhoods['Borough'][neighborhood_na]
neighborhoods = neighborhoods.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(x))
neighborhoods

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
neighborhoods.shape

(103, 3)

PART TWO
Use the Geocoder package or the csv file to create the a dataframe

In [7]:
lat = []
lng = []
lat_lng_coords = None
postal_code = neighborhoods['Postcode']
for pc in postal_code:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
    lat_lng_coords = g.latlng
    lat.append(lat_lng_coords[0])
    lng.append(lat_lng_coords[1])


In [9]:
nh_complete = neighborhoods
nh_complete['Latitude'] = lat
nh_complete['Longitude'] = lng
nh_complete

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726276,-79.263625
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713054,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724235,-79.227925
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696770,-79.259967
